In [2]:
import os
import pandas as pd
import json
import re
from pathlib import Path

def parse_qa_file(file_path: str) -> list:
    """
    Парсит файл формата Q:/A: в список словарей
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read().strip()
    
    qa_pairs = []
    current_q = None
    
    for line in content.split('\n'):
        line = line.strip()
        if not line:
            continue
            
        if line.startswith('Q:'):
            if current_q is not None:
                qa_pairs.append({"question": current_q, "answer": current_a})
            current_q = line[2:].strip()
            current_a = ""
        elif line.startswith('A:'):
            current_a = line[2:].strip()
    
    if current_q is not None:
        qa_pairs.append({"question": current_q, "answer": current_a})
    
    return qa_pairs

def csv_to_table_dict(csv_path: str) -> dict:
    """
    Конвертирует CSV файл в формат таблицы для JSON
    """
    # Читаем CSV с сохранением всех типов данных как строк
    df = pd.read_csv(csv_path, dtype=str)
    
    # Название таблицы - имя файла без расширения
    table_title = Path(csv_path).stem
    
    # Форматируем заголовки
    headers = [[col, ""] for col in df.columns.tolist()]
    
    # Форматируем данные
    data = []
    for _, row in df.iterrows():
        row_data = []
        for col in df.columns:
            cell_value = str(row[col]).strip() if pd.notna(row[col]) else ""
            # Все ячейки без ссылок для pure multitable режима
            row_data.append([cell_value, []])
        data.append(row_data)
    
    return {
        "title": table_title,
        "header": headers,
        "data": data
    }

def create_simple_mapping(qa_pairs: list, csv_files: list) -> list:
    """
    Создает простое сопоставление: один вопрос = одна таблица (по порядку)
    """
    mappings = []
    
    if len(qa_pairs) != len(csv_files):
        print(f"Внимание: количество вопросов ({len(qa_pairs)}) не совпадает с количеством таблиц ({len(csv_files)})")
        print("Будет использовано сопоставление по порядку для первых", min(len(qa_pairs), len(csv_files)), "элементов")
    
    for i in range(min(len(qa_pairs), len(csv_files))):
        mappings.append({
            "question_data": qa_pairs[i],
            "tables": [csv_files[i]]
        })
    
    return mappings

def create_smart_mapping(qa_pairs: list, csv_files: list, tables_dir: str) -> list:
    """
    Создает "умное" сопоставление вопросов и таблиц на основе анализа содержимого
    """
    mappings = []
    
    # Читаем содержимое всех таблиц для анализа
    table_contents = {}
    for csv_file in csv_files:
        file_path = os.path.join(tables_dir, csv_file)
        df = pd.read_csv(file_path, nrows=5)  # Читаем только первые 5 строк для анализа
        table_contents[csv_file] = {
            "columns": df.columns.tolist(),
            "sample_data": df.values.flatten().tolist()[:10]  # Первые 10 значений
        }
    
    # Сопоставляем каждый вопрос с наиболее релевантной таблицей
    for qa in qa_pairs:
        question = qa["question"].lower()
        best_match = None
        best_score = 0
        
        for csv_file, content in table_contents.items():
            score = 0
            
            # Анализируем соответствие по названию файла
            filename_keywords = re.split(r'[\s_\-]+', Path(csv_file).stem.lower())
            for keyword in filename_keywords:
                if keyword in question:
                    score += 2
            
            # Анализируем соответствие по заголовкам колонок
            for col in content["columns"]:
                col_clean = re.sub(r'[^\w\s]', '', col.lower())
                if col_clean in question:
                    score += 3
            
            # Анализируем соответствие по данным таблицы
            for value in content["sample_data"]:
                if pd.isna(value) or not isinstance(value, str):
                    continue
                value_clean = re.sub(r'[^\w\s]', '', str(value).lower())
                if len(value_clean) > 3 and value_clean in question:
                    score += 1
            
            if score > best_score:
                best_score = score
                best_match = csv_file
        
        if best_match and best_score > 0:
            mappings.append({
                "question_data": qa,
                "tables": [best_match]
            })
        else:
            # Если не нашли подходящую таблицу, используем первую доступную
            mappings.append({
                "question_data": qa,
                "tables": [csv_files[0]] if csv_files else []
            })
    
    return mappings

def create_multitable_mapping(qa_pairs: list, csv_files: list) -> list:
    """
    Создает сопоставление для multitable-сценария: один вопрос = все таблицы
    """
    return [{
        "question_data": qa,
        "tables": csv_files.copy()
    } for qa in qa_pairs]

def generate_json_files(
    qa_file: str, 
    tables_dir: str, 
    output_dir: str,
    mode: str = "smart"  # simple, smart, multitable
):
    """
    Основная функция генерации JSON файлов
    """
    # Создаем выходную директорию
    os.makedirs(output_dir, exist_ok=True)
    
    # Парсим вопросы и ответы
    qa_pairs = parse_qa_file(qa_file)
    if not qa_pairs:
        print("Не найдено вопросов в файле!")
        return
    
    print(f"Найдено вопросов: {len(qa_pairs)}")
    
    # Получаем список CSV файлов
    csv_files = [f for f in os.listdir(tables_dir) if f.endswith('.csv')]
    if not csv_files:
        print("Не найдено CSV файлов в директории!")
        return
    
    print(f"Найдено таблиц: {len(csv_files)}")
    
    # Выбираем режим сопоставления
    if mode == "simple":
        mappings = create_simple_mapping(qa_pairs, csv_files)
    elif mode == "multitable":
        mappings = create_multitable_mapping(qa_pairs, csv_files)
    else:  # smart mode (по умолчанию)
        mappings = create_smart_mapping(qa_pairs, csv_files, tables_dir)
    
    # Генерируем JSON файлы
    for i, mapping in enumerate(mappings):
        question = mapping["question_data"]["question"]
        tables = mapping["tables"]
        
        if not tables:
            print(f"Пропущен вопрос '{question}': нет подходящих таблиц")
            continue
        
        # Обрабатываем одну или несколько таблиц
        table_data = []
        for table_file in tables:
            csv_path = os.path.join(tables_dir, table_file)
            table_dict = csv_to_table_dict(csv_path)
            table_data.append(table_dict)
        
        # Формируем итоговый JSON
        if len(tables) == 1:
            # Single-table режим
            output_json = {
                "question": question,
                "table": table_data[0],
                # Для pure multitable режима поле "links" отсутствует
            }
        else:
            # Multitable режим
            output_json = {
                "question": question,
                "tables": table_data,  # Массив таблиц
                # Для pure multitable режима поле "links" отсутствует
            }
        
        # Сохраняем в файл
        safe_question = re.sub(r'[^\w\s\-]', '', question)[:50].strip().replace(' ', '_')
        output_path = os.path.join(output_dir, f"q{i+1}_{safe_question}.json")
        
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(output_json, f, ensure_ascii=False, indent=2)
        
        print(f"Создан файл: {os.path.basename(output_path)}")
        print(f"  Вопрос: {question}")
        print(f"  Таблицы: {', '.join(tables)}")
        print()
    
    print(f"Всего создано JSON файлов: {len(mappings)}")
    print(f"Файлы сохранены в: {os.path.abspath(output_dir)}")

In [3]:
import sqlite3
import pandas as pd

db_path = "/home/george/Documents/code/vkr/rustbench/tables-qa/set/northwind/3/local_snapshot.db"
conn = sqlite3.connect(db_path)

# Получаем список всех таблиц
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

for table_name in tables:
    table_name = table_name[0]
    table = pd.read_sql_query(f'SELECT * FROM "{table_name}"', conn)
    table.to_csv(f"/home/george/Documents/code/vkr/HybridGraphs/json_inputs/northwind3-sample/tables/{table_name}.csv", index=False)

conn.close()

In [8]:
tables_dir = "json_inputs/northwind3-mini/tables/"

for name in os.listdir(tables_dir):
    pd.read_csv(tables_dir + name).iloc[:2].to_csv(tables_dir + name)


In [4]:
QA_FILE = "json_inputs/northwind3-sample/QA.txt"          # Путь к файлу с вопросами
TABLES_DIR = "json_inputs/northwind3-sample/tables"              # Папка с CSV файлами
OUTPUT_DIR = "json_inputs"         # Папка для выходных JSON файлов
MODE = "multitable"                     # Режим сопоставления: simple, smart, multitable

generate_json_files(
    qa_file=QA_FILE,
    tables_dir=TABLES_DIR,
    output_dir=OUTPUT_DIR,
    mode=MODE
)

Найдено вопросов: 1
Найдено таблиц: 30
Создан файл: q1_How_many_unique_orders_were_placed_by_US_customers.json
  Вопрос: How many unique orders were placed by US customers that were processed by Seattle-based employees?
  Таблицы: order subtotals.csv, sqlite_sequence.csv, category sales for 1997.csv, summary of sales by year.csv, products above average price.csv, orders.csv, employees.csv, employeeterritories.csv, shippers.csv, alphabetical list of products.csv, order details.csv, products.csv, sales totals by amount.csv, orders qry.csv, order details extended.csv, current product list.csv, product sales for 1997.csv, sales by category.csv, customercustomerdemo.csv, products by category.csv, invoices.csv, categories.csv, suppliers.csv, customer and suppliers by city.csv, quarterly orders.csv, customerdemographics.csv, territories.csv, summary of sales by quarter.csv, customers.csv, region.csv

Всего создано JSON файлов: 1
Файлы сохранены в: /home/george/Documents/code/vkr/HybridGraphs/